In [11]:
import pandas as pd
import os

class SimulationTimeScaler:
    def __init__(self, num_frames: int, num_timesteps: int, timestep_length_ns: float):
        """
        Initialize with the number of frames, total timesteps, and the length of each timestep in nanoseconds.
        :param num_frames: Total number of frames in the simulation.
        :param num_timesteps: Total number of timesteps in the simulation.
        :param timestep_length_ns: Length of one timestep in nanoseconds.
        """
        self.num_frames = num_frames
        self.num_timesteps = num_timesteps
        self.timestep_length_ns = timestep_length_ns
        self.timesteps_per_frame = num_timesteps / num_frames

    def calculate_length_from_timesteps(self) -> float:
        """
        Calculate the total length of the simulation in nanoseconds based on the number of timesteps.
        :return: Total length of the simulation in nanoseconds.
        """
        return self.num_timesteps * self.timestep_length_ns

    def calculate_timesteps(self, simulation_length_ns: float) -> int:
        """
        Calculate the number of timesteps for a given length of simulation in nanoseconds.
        :param simulation_length_ns: The total length of the simulation in nanoseconds.
        :return: The number of timesteps.
        """
        return int(simulation_length_ns / self.timestep_length_ns)

    def scale_to_nanoseconds(self, dataset: pd.DataFrame, x_column: str, new_column_name: str) -> pd.DataFrame:
        """
        Scale the x-column of the dataset (assumed to be frame numbers) to nanoseconds and add as a new column.
        :param dataset: A pandas DataFrame with at least one column for frame numbers.
        :param x_column: The name of the column to scale (the frame numbers).
        :param new_column_name: The name of the new column to be added for scaled values.
        :return: A new DataFrame with the added scaled x-column (in nanoseconds).
        """
        dataset_scaled = dataset.copy()
        dataset_scaled[new_column_name] = dataset[x_column] * self.timesteps_per_frame * self.timestep_length_ns
        return dataset_scaled

    @staticmethod
    def load_data(filename: str, column_names: list, skiprows: int = 0) -> pd.DataFrame:
        """
        Load data from a .txt or .csv file into a pandas DataFrame with custom column names.
        
        :param filename: Path to the file.
        :param column_names: List of column names to assign to the data.
        :param skiprows: Number of rows to skip at the start of the file (default is 0).
        :return: A pandas DataFrame with the loaded data and custom column names.
        """
        if filename.endswith('.txt'):
            return pd.read_csv(filename, sep=r'\s+', skiprows=skiprows, names=column_names)
        elif filename.endswith('.csv'):
            return pd.read_csv(filename, skiprows=skiprows, names=column_names)
        else:
            raise ValueError("Unsupported file format. Please provide a .txt or .csv file.")

    @staticmethod
    def save_data(data: pd.DataFrame, output_dir: str, output_filename: str) -> str:
        """
        Save the modified dataset to the specified directory.
        
        :param data: DataFrame to be saved.
        :param output_dir: Directory where the file will be saved.
        :param output_filename: The name of the file to save (with extension).
        :return: Full path to the saved file.
        """
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, output_filename)
        data.to_csv(output_path, index=False)
        return output_path


In [12]:
# Define your parameters
num_frames = 1000  # Total number of frames in the simulation
num_timesteps = 50000000  # 50 million timesteps
timestep_length_ns = 0.002  # e.g., 2 picoseconds = 0.002 nanoseconds
simulation_time_ns = 100000  # Example: 1 microsecond (1,000,000 nanoseconds)

# Create an instance of SimulationTimeScaler
scaler = SimulationTimeScaler(num_frames, num_timesteps, timestep_length_ns)

# Calculate simulation length in nanoseconds using the number of timesteps
total_length_from_timesteps = scaler.calculate_length_from_timesteps()
print(f"Total simulation length for {num_timesteps} timesteps of {timestep_length_ns} ns: {total_length_from_timesteps} ns")

# Calculate the number of timesteps for a given length of time
timesteps = scaler.calculate_timesteps(simulation_time_ns)
print(f"Number of timesteps for {simulation_time_ns} ns: {timesteps}")

# Calculate and print the length of time per frame
time_per_frame_ns = scaler.timestep_length_ns * scaler.timesteps_per_frame
print(f"Length of time per frame: {time_per_frame_ns} ns")

Total simulation length for 50000000 timesteps of 0.002 ns: 100000.0 ns
Number of timesteps for 100000 ns: 50000000
Length of time per frame: 100.0 ns


In [ ]:
# Define your custom column names
column_names = ['Frame', 'HBonds']  # Replace these with your actual column names
    
# Load the data from a .txt or .csv file
filename = 'your_data.txt'  # Replace with your actual file path
skiprows = 1  # Number of rows to skip at the top (if any), default 0
data = SimulationTimeScaler.load_data(filename, column_names, skiprows)

# Scale the dataset
x_column = "Frame"  # Replace with your actual column name
new_column_name = "Frame_ns"  # Name of the new column to be added
scaled_data = scaler.scale_to_nanoseconds(data, x_column, new_column_name)

# Specify the directory and filename to save the new dataset
output_dir = 'output_directory'  # Replace with your desired directory
output_filename = 'scaled_data.csv'  # Replace with the desired filename
output_path = SimulationTimeScaler.save_data(scaled_data, output_dir, output_filename)
print(f"Scaled dataset saved to: {output_path}")